In [1]:
"""
Author: Fiqri Wicaksono (fiqriwicaksono@gmail.com)
"""

# import library yang diperlukan
import pandas as pd
import numpy as np
import os
from os.path import join
from datetime import datetime, date, timedelta

In [2]:
# import cufflink library
# please change the notebook state into trusted unless the plots won't show
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

In [ ]:
# if you are using google colab please run the following code
# import plotly.io as pio 
# pio.renderers.default = 'colab'

# Get Data

In [2]:
# path ke file yang diperlukan
flight_dir = join(os.getcwd(), "Flight")
supp_dir = join(os.getcwd(), "Supplementary")

# 1. Jakarta Flight

In [3]:
source_path = join(flight_dir, "landed_flight_data_indonesia_departures (CGK and HALIM).xlsx")

In [4]:
jkt_flight = pd.read_excel(source_path, index=False)

In [5]:
# Cek informasi data yang diambil
jkt_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79607 entries, 0 to 79606
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          79607 non-null  int64 
 1   date                79607 non-null  object
 2   flight_tail_number  79607 non-null  object
 3   from                79607 non-null  object
 4   to                  79607 non-null  object
 5   aircraft            79607 non-null  object
 6   std                 79607 non-null  object
 7   sta                 79607 non-null  object
 8   status              79607 non-null  object
dtypes: int64(1), object(8)
memory usage: 5.5+ MB


In [6]:
jkt_flight.head(10)

Unnamed: 0         date flight_tail_number           from              to  \
0          12  24 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
1          15  21 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
2          16  20 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
3          17  19 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
4          18  18 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
5          19  17 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
6          20  16 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
7          21  15 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
8          22  14 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   
9          23  13 Mar 2020             QZ7520  Jakarta (CGK)  Denpasar (DPS)   

        aircraft      std       sta           status  
0  A320 (PK-AZN)  9:25 AM  12:25 PM  Landed 11:50 AM  
1  A320 (PK-AZF)  9:25 AM  12:25 PM  Landed 12:08 PM  
2  A320 (PK-AXE)  9:25 AM  12:25 PM  Landed 12:03 PM  
3  A320 (PK-AXT)  9:25 AM  12:25 PM  Landed 11:54 AM  
4  A320 (PK-AZF)  9:25 AM  12:25 PM  Landed 12:18 PM  
5  A320 (PK-AZO)  9:25 AM  12:25 PM  Landed 12:04 PM  
6  A320 (PK-AXU)  9:25 AM  12:25 PM  Landed 12:06 PM  
7  A320 (PK-AZN)  9:25 AM  12:25 PM  Landed 12:01 PM  
8  A320 (PK-AZD)  9:25 AM  12:25 PM  Landed 12:00 PM  
9  A320 (PK-AXT)  9:25 AM  12:25 PM  Landed 12:02 PM

# 2. Aircraft Capacity Dataset

In [7]:
source_path = join(supp_dir, "aircraft_capacity.xlsx")
aircraft_capacity = pd.read_excel(source_path)

In [8]:
# Cek informasi data yang diambil
aircraft_capacity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aircraft  26 non-null     object
 1   capacity  26 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 544.0+ bytes


In [9]:
aircraft_capacity

aircraft  capacity
0      B739       215
1      B738       162
2      A320       186
3      A20N       180
4      B733       149
5      A339       287
6      B735       132
7      CRJX       100
8      A332       250
9      A333       277
10      73F       269
11      738       162
12      737       177
13      735       126
14     B77W       368
15     A359       293
16     A21N       244
17     A321       200
18     B788       240
19     B789       215
20     B773       244
21     B772       375
22     B78X       294
23     B763       216
24     AT75        72
25     AT76        78

# Data Preprocessing

In [17]:
# create checkpoint
df_jkt = jkt_flight.copy()

In [18]:
# membuat function transformasi
def transform(df, capacity):
    # Take only the necessary columns
    df = df[['date', 'from', 'to', 'aircraft']]
    # Convert date to date format
    df['date'] = pd.to_datetime(df['date']).dt.date
    # Take the necessary timeline
    df = df.loc[df['date']>=date(2020, 3, 2)]
    # Take only the city name from the 'from' and 'to' columns
    df['from'] = [x.split()[0] for x in df['from']]
    df['to'] = [x.split()[0] for x in df['to']]
    # Take only flights from Jakarta and remove flight to unknown location
    df = df.loc[(df['from']=='Jakarta') & (df['to']!='-')]
    # Get only aircraft type
    df['aircraft'] = [x.split()[0] for x in df['aircraft']]
    # Merge to get capacity
    df = df.merge(capacity, on='aircraft')
    return df

In [19]:
df = transform(df_jkt, aircraft_capacity)

In [20]:
# agregasi berdasarkan jumlah penerbangan
df_agg = df.groupby(['from', 'to'], as_index=False)['aircraft'].count().sort_values('aircraft', ascending=False)\
            .reset_index(drop=True)
df_agg = df_agg.rename(columns={'aircraft':'total_aircraft'})
df_agg

from               to  total_aircraft
0   Jakarta         Surabaya             829
1   Jakarta       Yogyakarta             642
2   Jakarta            Medan             598
3   Jakarta         Denpasar             549
4   Jakarta         Semarang             380
5   Jakarta           Padang             304
6   Jakarta        Palembang             303
7   Jakarta       Balikpapan             297
8   Jakarta        Pekanbaru             293
9   Jakarta         Makassar             276
10  Jakarta           Bandar             260
11  Jakarta        Pontianak             222
12  Jakarta            Batam             208
13  Jakarta            Jambi             199
14  Jakarta        Surakarta             197
15  Jakarta          Pangkal             189
16  Jakarta           Malang             188
17  Jakarta      Banjarmasin             172
18  Jakarta        Singapore             168
19  Jakarta           Manado             152
20  Jakarta          Tanjung             129
21  Jakarta           Lombok             129
22  Jakarta          Bandung             109
23  Jakarta         Bengkulu             102
24  Jakarta        Samarinda              92
25  Jakarta            Kuala              71
26  Jakarta            Banda              60
27  Jakarta     Palangkaraya              53
28  Jakarta     Blimbingsari              50
29  Jakarta  Siborong-Borong              48
30  Jakarta     Lubuklinggau              41
31  Jakarta           Labuan              40
32  Jakarta           Taipei              40
33  Jakarta          Bangkok              36
34  Jakarta          Kendari              33
35  Jakarta          Ternate              26
36  Jakarta             Doha              26
37  Jakarta             Palu              25
38  Jakarta            Ambon              25
39  Jakarta              Abu              23
40  Jakarta      Tasikmalaya              20
41  Jakarta           Sydney              19
42  Jakarta           Muscat              14
43  Jakarta            Perth              10
44  Jakarta           Penang               9
45  Jakarta           Manila               7
46  Jakarta           Sorong               2
47  Jakarta           Sampit               2
48  Jakarta                —               2

In [21]:
# bar chart berdasarkan 10 kota teratas
df_agg[:11].sort_values('total_aircraft').iplot(kind='bar',
                                                 x='to',
                                                 y='total_aircraft',
                                                 color='blue',
                                                 theme='henanigans',
                                                 orientation='h',
                                                 xTitle='Jumlah Penerbangan dari Jakarta',
                                                 yTitle='Menuju')

In [22]:
# agregasi berdasarkan waktu
df_waktu = df.groupby('date', as_index=False)['aircraft'].count()
df_waktu = df_waktu.rename(columns={'aircraft':'total_aircraft'})
df_waktu

date  total_aircraft
0   2020-03-02             328
1   2020-03-03             315
2   2020-03-04             309
3   2020-03-05             316
4   2020-03-06             344
5   2020-03-07             321
6   2020-03-08             340
7   2020-03-09             319
8   2020-03-10             318
9   2020-03-11             309
10  2020-03-12             312
11  2020-03-13             335
12  2020-03-14             328
13  2020-03-15             355
14  2020-03-16             319
15  2020-03-17             320
16  2020-03-18             309
17  2020-03-19             296
18  2020-03-20             289
19  2020-03-21             251
20  2020-03-22             245
21  2020-03-23             226
22  2020-03-24             214
23  2020-03-25             210
24  2020-03-26             199
25  2020-03-27             205
26  2020-03-28              37

In [23]:
# line chart berdasarkan waktu
# tanggal 15 Maret 2020 merupakan hari dimana pemerintah menghimbau untuk social distancing
df_waktu.iplot(mode='lines',
               x='date',
               y='total_aircraft', vline='2020-03-15',
               color='blue', theme='henanigans',
               title='Jumlah Penerbangan Dari Jakarta')

# Analisis Jumlah Penumpang

In [24]:
# mengambil 10 kota teratas yang dituju dari Jakarta
target_city = "Surabaya|Yogyakarta|Medan|Denpasar|Semarang|Padang|Palembang|Balikpapan|Pekanbaru|Makassar|Bandar"
df_top10 = df.loc[df['to'].str.contains(target_city)]
df_top10

date     from          to aircraft  capacity
0     2020-03-24  Jakarta    Denpasar     A320       186
1     2020-03-21  Jakarta    Denpasar     A320       186
2     2020-03-20  Jakarta    Denpasar     A320       186
3     2020-03-19  Jakarta    Denpasar     A320       186
4     2020-03-18  Jakarta    Denpasar     A320       186
...          ...      ...         ...      ...       ...
7635  2020-03-11  Jakarta  Balikpapan     B733       149
7636  2020-03-10  Jakarta  Balikpapan     B733       149
7637  2020-03-06  Jakarta  Balikpapan     B733       149
7638  2020-03-04  Jakarta  Balikpapan     B733       149
7639  2020-03-03  Jakarta  Balikpapan     B733       149

[4731 rows x 5 columns]

In [26]:
# menambahkan perkiraan penumpang 70% dari kapasitas pesawat
df_top10["total_passenger"] = round((df_top10["capacity"] * 0.7), 0)
df_top10

date     from          to aircraft  capacity  total_passenger
0     2020-03-24  Jakarta    Denpasar     A320       186            130.0
1     2020-03-21  Jakarta    Denpasar     A320       186            130.0
2     2020-03-20  Jakarta    Denpasar     A320       186            130.0
3     2020-03-19  Jakarta    Denpasar     A320       186            130.0
4     2020-03-18  Jakarta    Denpasar     A320       186            130.0
...          ...      ...         ...      ...       ...              ...
7635  2020-03-11  Jakarta  Balikpapan     B733       149            104.0
7636  2020-03-10  Jakarta  Balikpapan     B733       149            104.0
7637  2020-03-06  Jakarta  Balikpapan     B733       149            104.0
7638  2020-03-04  Jakarta  Balikpapan     B733       149            104.0
7639  2020-03-03  Jakarta  Balikpapan     B733       149            104.0

[4731 rows x 6 columns]

In [27]:
# aggregasi berdasarkan kota
df_pass = df_top10.groupby(['from', 'to'], as_index=False)['total_passenger'].sum()\
                    .sort_values('total_passenger', ascending=False).reset_index(drop=True)
df_pass

from          to  total_passenger
0   Jakarta    Surabaya         103307.0
1   Jakarta       Medan          80060.0
2   Jakarta  Yogyakarta          77742.0
3   Jakarta    Denpasar          75472.0
4   Jakarta    Semarang          48861.0
5   Jakarta      Padang          40328.0
6   Jakarta  Balikpapan          38646.0
7   Jakarta   Palembang          37611.0
8   Jakarta   Pekanbaru          36908.0
9   Jakarta    Makassar          34917.0
10  Jakarta      Bandar          29718.0

In [28]:
# bar chart jumlah penumpang berdasarkan 10 kota teratas
df_pass.sort_values('total_passenger').iplot(kind='bar',
                                                 x='to',
                                                 y='total_passenger',
                                                 color='blue',
                                                 theme='henanigans',
                                                 orientation='h',
                                                 xTitle='Jumlah Penumpang dari Jakarta',
                                                 yTitle='Menuju')

In [29]:
# agregasi berdasarkan waktu
df_date = df_top10.groupby(['date'], as_index=False)['total_passenger'].sum()
df_date

date  total_passenger
0   2020-03-02          25144.0
1   2020-03-03          24574.0
2   2020-03-04          24189.0
3   2020-03-05          24891.0
4   2020-03-06          26803.0
5   2020-03-07          24934.0
6   2020-03-08          27089.0
7   2020-03-09          25480.0
8   2020-03-10          25022.0
9   2020-03-11          24602.0
10  2020-03-12          25073.0
11  2020-03-13          26852.0
12  2020-03-14          25511.0
13  2020-03-15          28299.0
14  2020-03-16          24793.0
15  2020-03-17          24930.0
16  2020-03-18          24366.0
17  2020-03-19          23564.0
18  2020-03-20          23374.0
19  2020-03-21          20018.0
20  2020-03-22          19784.0
21  2020-03-23          17907.0
22  2020-03-24          16938.0
23  2020-03-25          15416.0
24  2020-03-26          16003.0
25  2020-03-27          15603.0
26  2020-03-28           2411.0

In [30]:
# line chart jumlah penumpang berdasarkan waktu
# tanggal 15 Maret 2020 merupakan hari dimana pemerintah menghimbau untuk social distancing
df_date.iplot(mode='lines',
               x='date',
               y='total_passenger', vline='2020-03-15',
               color='blue', theme='henanigans',
               title='Jumlah Penumpang Dari Jakarta')